In [1]:
import sys
sys.path.append('..')
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sfp_nsdsyn import *
import warnings
import itertools
import matplotlib as mpl

warnings.filterwarnings("ignore", category=RuntimeWarning)
pd.options.mode.chained_assignment = None
%load_ext autoreload
%autoreload 2

In [7]:
output_dir='/Volumes/server/Projects/sfp_nsd/derivatives'
results_dir = os.path.join(output_dir, 'sfp_model', 'results_2D')
fig_dir = os.path.join(output_dir, 'figures', 'sfp_model', 'results_2D')
lr = 0.0005
max_epoch = 30000
dset='nsdsyn'
roi_list = ['V1','V2','V3']
vsc='pRFsize'
nsd_subj_list = [utils.sub_number_to_string(sn, 'nsdsyn') for sn in np.arange(1,9)]
broderick_sn_list = [1, 6, 7, 45, 46, 62, 64, 81, 95, 114, 115, 121]
broderick_subj_list = [utils.sub_number_to_string(i, 'broderick') for i in broderick_sn_list]
ARGS_2D = ['sub','lr','eph','roi','dset']

# NSD 
## model

In [8]:
pt_file_list = []
dset = 'nsdsyn'
for subj, roi in itertools.product(nsd_subj_list, ['V1','V2','V3']):
    file_name = os.path.join(results_dir, dset, f'model-params_lr-{lr}_eph-{max_epoch}_dset-{dset}_sub-{subj}_roi-{roi}_vs-{vsc}.pt')
    pt_file_list.append(file_name)
nsd_params = model.load_all_models(pt_file_list, *ARGS_2D)

## NSD precision

In [9]:
nsd_precision_s =  pd.read_csv(os.path.join(output_dir, 'dataframes', dset, 'precision', f'precision-s_dset-{dset}_vs-pRFsize.csv'))
nsd_df = pd.merge(nsd_params, nsd_precision_s[['sub','vroinames','precision']], on=['sub','vroinames'])

# Broderick et al
## model

In [10]:
pt_file_list = []
dset = 'broderick'
tfunc_types=['corrected']
for subj, roi, tfunc in itertools.product(broderick_subj_list, ['V1'], tfunc_types):
    file_name = os.path.join(results_dir, dset, f'tfunc-{tfunc}_model_lr-{lr}_eph-{max_epoch}_dset-{dset}_sub-{subj}_roi-{roi}_vs-{vsc}.pt')
    pt_file_list.append(file_name)
broderick_params = model.load_all_models(pt_file_list, *ARGS_2D+['tfunc'])
broderick_params.replace({'tfunc': {'derick/tfunc corrected': 'corrected', 'derick/tfunc uncorrected': 'uncorrected'}}, inplace=True)

## broderick precision

In [11]:
broderick_precision_s =  pd.read_csv(os.path.join(output_dir, 'dataframes', dset, 'precision', f'precision-s_dset-{dset}_vs-pRFsize.csv'))
all_broderick_df = pd.merge(broderick_params, broderick_precision_s[['sub','vroinames','precision']], on=['sub','vroinames'])

In [12]:
broderick_df = all_broderick_df.query('tfunc == "corrected"')
broderick_df['dset_type'] = 'Broderick et al. V1'
nsd_df['dset_type'] = nsd_df['vroinames'].apply(lambda x: f'NSD {x}')
final_df = pd.concat((broderick_df, nsd_df), axis=0)

In [13]:
final_df['goal'] = final_df['vroinames'].apply(lambda x: 'Replication' if x == "V1" else 'Extension')
tmp = nsd_df.query('vroinames == "V1"')
tmp['goal'] = 'Extension'
final_df = pd.concat((tmp, final_df), axis=0)


In [13]:
utils.calculate_weighted_mean(final_df, ['A_1','A_2'], 'precision', groupby=['dset_type'])


,dset_type,A_1,A_2
0,Broderick et al. V1,0.011701,0.000306
1,NSD V1,0.017979,-0.011636
2,NSD V2,-0.001070,-0.018698
3,NSD V3,0.010500,-0.011148


# Plot parameter estimates

In [14]:
utils.calculate_weighted_mean(final_df, ['p_1','p_2'], 'precision', groupby=['dset_type'])


,dset_type,p_1,p_2
0,Broderick et al. V1,0.023768,-0.024918
1,NSD V1,0.052048,-0.028581
2,NSD V2,-0.017168,-0.071220
3,NSD V3,-0.044421,-0.034815
